EJERCICIO 3 
Trabajo Final:
“Datos masivos y minería de datos”


Profesor:  David Diaz
Integrantes:
Cristian Hinojosa,
Pablo Chaparro

# Instalando Spark y sus pre-requisitos

Spark y sus prerequisitos no están preinstalados en GoogleColab.
Por lo tanto, el primer paso es instalarlos. El principal pre-requisito es java y opcional 
hadoop.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark #esta libreria encuentra spark dentro de Colab y le permite usarlo más adelante

El siguiente paso es definir las variables de entorno

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!ls #podemos listar los archivos instalables que se descargaron

cod_comunas.csv		      spark-3.1.1-bin-hadoop3.2
rol_agricola.csv	      spark-3.1.1-bin-hadoop3.2.tgz
rol_no_agricola.csv	      spark-3.1.1-bin-hadoop3.2.tgz.1
sample_data		      spark-3.1.1-bin-hadoop3.2.tgz.2
SII_roles_avaluo_table.zip    spark-warehouse
SII_roles_avaluo_table.zip.1  terreno_constr_no_agri.csv


# Configurando la Sesión y lanzando Spark

Podemos comenzar a utilizar spark. Creamos una SparkSession.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
.master("local[2]")\
.appName("Colab")\
.config('spark.driver.memory', '10g')\
.config('spark.ui.port', '4040')\
.getOrCreate()

In [ ]:
""" ejemplo dynamic allocation of executors :
spark.conf.set("spark.dynamicAllocation.enabled", "true")
spark.conf.set("spark.executor.cores", 4)
spark.conf.set("spark.dynamicAllocation.minExecutors","1")
spark.conf.set("spark.dynamicAllocation.maxExecutors","5")
"""

' ejemplo dynamic allocation of executors :\nspark.conf.set("spark.dynamicAllocation.enabled", "true")\nspark.conf.set("spark.executor.cores", 4)\nspark.conf.set("spark.dynamicAllocation.minExecutors","1")\nspark.conf.set("spark.dynamicAllocation.maxExecutors","5")\n'

In [ ]:
spark #hello I'm spark

# **TAREA EJERCICIO 3 SPARK - SQL**

In [ ]:
!wget https://www.dropbox.com/s/ianxkffu14dp0ru/SII_roles_avaluo_table.zip #archivos tabulares grandes aprox 7 millones de filas

--2023-02-09 21:28:42--  https://www.dropbox.com/s/ianxkffu14dp0ru/SII_roles_avaluo_table.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/ianxkffu14dp0ru/SII_roles_avaluo_table.zip [following]
--2023-02-09 21:28:42--  https://www.dropbox.com/s/raw/ianxkffu14dp0ru/SII_roles_avaluo_table.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6598a1d1a6f168eb3d8528c699.dl.dropboxusercontent.com/cd/0/inline/B2PbnO1I2_eXYstxs_whwKgyDaPpBvTP8KOLj1euP2kK5UH77B_Oc-NwcQorMne5LdQs--STArAVDzs6GgyYRfkAyQYx65bYU31e-XQ_ZguA8i_V116DwmhPRb2niwy3B5aObss8KWlw389dirduGZB8UmDId3nMZZYll-cUWyHI4w/file# [following]
--2023-02-09 21:28:42--  https://uc6598a1d1a6f168eb3d8528c699.dl.dropboxusercontent.com/cd/0/inline/B2PbnO1I2_eXYstxs_whwKgyDaPpBvTP8KOLj1

In [ ]:
!unzip -o SII_roles_avaluo_table.zip

Archive:  SII_roles_avaluo_table.zip
  inflating: terreno_constr_no_agri.csv  
  inflating: rol_agricola.csv        
  inflating: rol_no_agricola.csv     
  inflating: cod_comunas.csv         


In [ ]:
!ls

cod_comunas.csv      sample_data		 spark-3.1.1-bin-hadoop3.2.tgz
rol_agricola.csv     SII_roles_avaluo_table.zip  spark-warehouse
rol_no_agricola.csv  spark-3.1.1-bin-hadoop3.2	 terreno_constr_no_agri.csv


## CREACIÓN DE TABLAS

In [ ]:
cod_comunas_sp = spark.read.csv("cod_comunas.csv",header=True, inferSchema=True)

In [ ]:
rol_agricola_sp = spark.read.csv("rol_agricola.csv",header=True, inferSchema=True)

In [ ]:
rol_no_agricola_sp = spark.read.csv("rol_no_agricola.csv",header=True, inferSchema=True)

In [ ]:
rol_no_agricola_sp.createOrReplaceTempView("rol_no_agricola_sql")

In [ ]:
terreno_constr_no_agri_sp = spark.read.csv("terreno_constr_no_agri.csv",header=True, inferSchema=True)

In [ ]:
terreno_constr_no_agri_sp.createOrReplaceTempView("terreno_const_sql")

In [ ]:
rol_no_agricola_sp.show()

In [ ]:
terreno_constr_no_agri_sp.show()

+---+----------+----------+----------+-------------+-------------------------+-----------------------+-----------------+----------------------+---------------------+------------------------------+
|_c0|cod_comuna|nr_manzana|nr_predial|corr_lin_prod|cod_material_estruc_const|cod_calidad_linea_const|anio_linea_constr|sup_linea_constr_m2/m3|cod_dest_linea_constr|cod_cond_especial_linea_constr|
+---+----------+----------+----------+-------------+-------------------------+-----------------------+-----------------+----------------------+---------------------+------------------------------+
|  0|      1101|         1|         4|            1|                       B |                      4|             1925|                   631|                    O|                          null|
|  1|      1101|         1|         4|            2|                       E |                      3|             1925|                   201|                    O|                          null|
|  2|      1101

## GROUP BY

In [ ]:
# creacion de tabla intermedia con la cantidad de materiales usados en la construccion por comuna y rol
construidos="""
SELECT cod_comuna,nr_manzana,nr_predial, count (distinct  cod_material_estruc_const) as materiales
FROM terreno_const_sql 
group by cod_comuna,nr_manzana,nr_predial
ORDER BY materiales DESC
"""
construidossql=spark.sql(construidos)
construidossql.show()
construidossql.createOrReplaceTempView("construidossql_view")


+----------+----------+----------+----------+
|cod_comuna|nr_manzana|nr_predial|materiales|
+----------+----------+----------+----------+
|      9201|      3204|        18|        12|
|      8210|     15006|        45|        12|
|     14166|       900|         1|        12|
|     14166|       499|         1|        12|
|     14111|      2910|         1|        12|
|      8210|     15006|     70006|        12|
|      2206|      8500|         5|        11|
|      7301|       564|         1|        11|
|      8206|      7400|         1|        11|
|      5307|      3205|         6|        10|
|     16301|       534|         1|        10|
|      8405|       372|     70017|        10|
|      8210|     15006|     70004|        10|
|     14114|      2113|         1|        10|
|      7101|       526|        48|        10|
|     14166|        36|        11|        10|
|     14109|       925|        12|        10|
|     10109|       546|       938|        10|
|     14109|       925|        14|

revisión de datos en tabla de propiedades no agricolas

In [ ]:
spark.sql("select count(*) as total_count from rol_no_agricola_sql").show()


+-----------+
|total_count|
+-----------+
|    7432738|
+-----------+



In [ ]:
#revisión de datos en tabla de materiales
spark.sql("select count(*) as total_count from construidossql_view").show()

+-----------+
|total_count|
+-----------+
|    6956192|
+-----------+



## LEFT JOIN

In [ ]:
#Left join de tabla de propiedades no agricolas con tabla de propiedades contruidas, se obtiene a traves del join la cantidad de materiales diferentes utilizados.
qry_join_rolnoagri_noagricontruido="""
SELECT A.*,B.materiales
FROM rol_no_agricola_sql as A
LEFT JOIN construidossql_view as B
ON A.cod_comuna = B.cod_comuna and A.nr_manzana = B.nr_manzana and A.nr_predial = B.nr_predial
ORDER BY A.avaluo_fisc_tot DESC
"""
rol_no_agricola_sql2=spark.sql(qry_join_rolnoagri_noagricontruido)
rol_no_agricola_sql2.show()
rol_no_agricola_sql2.createOrReplaceTempView("rol_no_agricola_sql_join")

+-------+----------+----------+----------+--------------------+---------------+----------------+--------------+----------------+---------------------------+----------------------------+--------------------------+---------------------------+----------------------------+--------------------------+-------------------------+----------+
|    _c0|cod_comuna|nr_manzana|nr_predial|     dir/nomb_predio|avaluo_fisc_tot|contri_semestral|cod_dest_princ|avaluo_exen_prop|cod_comuna_rol_bien_comun_1|nro_manzana_rol_bien_comun_1|nr_predio_rol_bien_comun_1|cod_comuna_rol_bien_comun_2|nro_manzana_rol_bien_comun_2|nr_predio_rol_bien_comun_2|sup_total_terreno_prod_m2|materiales|
+-------+----------+----------+----------+--------------------+---------------+----------------+--------------+----------------+---------------------------+----------------------------+--------------------------+---------------------------+----------------------------+--------------------------+-------------------------+----------

In [ ]:
# verificacion que left join mantiene la misma cantidad de registros de tabla inicial 
spark.sql("select count(*) as total_count from rol_no_agricola_sql_join").show()

+-----------+
|total_count|
+-----------+
|    7432738|
+-----------+



## WHERE CON 2 CONDICIONES

In [ ]:
#Propiedades en algarrobo de mas de 5000 m2 de superficie total y con avaluo fiscal menor a 10 millones.
qry_where="""
SELECT cod_comuna,nr_manzana,nr_predial,avaluo_fisc_tot,sup_total_terreno_prod_m2
FROM rol_no_agricola_sql_join 
where avaluo_fisc_tot<=10000000 and sup_total_terreno_prod_m2>5000 and cod_comuna='5406'
order by avaluo_fisc_tot desc
"""
noagricola_sql3=spark.sql(qry_where)
noagricola_sql3.show()
noagricola_sql3.createOrReplaceTempView("qry_where_view")

+----------+----------+----------+---------------+-------------------------+
|cod_comuna|nr_manzana|nr_predial|avaluo_fisc_tot|sup_total_terreno_prod_m2|
+----------+----------+----------+---------------+-------------------------+
|      5406|       267|       159|        9971683|                     5450|
|      5406|       267|       157|        9971683|                     5450|
|      5406|       267|       155|        9971683|                     5450|
|      5406|       267|       154|        9971683|                     5450|
|      5406|       267|       156|        9971683|                     5450|
|      5406|       267|       158|        9971683|                     5450|
|      5406|      3267|        15|        9929600|                     5427|
|      5406|      3267|        35|        9913134|                     5418|
|      5406|       237|        62|        9880199|                     5400|
|      5406|       570|        16|        9880199|                     5400|